In [24]:
import numpy as np
import pandas as pd
import pickle
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.model_selection import GridSearchCV,train_test_split,RandomizedSearchCV

In [2]:
def metricas(model,Xt,Xv,yt,yv):
    print "ROC train:%.3f | ROC test:%.3f " %(roc_auc_score(y_true=yt,y_score=model.predict_proba(Xt)[:,1]),roc_auc_score(y_true=yv,y_score=model.predict_proba(Xv)[:,1]))
    print "ACC train:%.3f | ACC test:%.3f " %(accuracy_score(y_true=yt,y_pred=model.predict(Xt)),accuracy_score(y_true=yv,y_pred=model.predict(Xv)))

In [3]:
df = pickle.load(open('churn_con_woe','rb'))

In [4]:
df.head()

,V_ACCT_LGHT,C_IP,C_VMP,V_N_VM_MESS,V_T_DAY_CHG,V_T_EVE_CALLS,V_T_EVE_CHG,V_T_NIG_CALLS,V_T_NIG_CHG,V_T_INT_CALLS,...,W_V_ACCT_LGHT,W_V_N_VM_MESS,W_V_T_DAY_CHG,W_V_T_EVE_CALLS,W_V_T_EVE_CHG,W_V_T_NIG_CALLS,W_V_T_NIG_CHG,W_V_T_INT_CALLS,W_V_T_INT_CHG,W_V_CS_CALLS
0,128,No,Yes,25,45.07,99,16.78,91,11.01,3,...,0.061033,1.219037,-1.864202,0.068283,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
1,156,No,No,0,29.67,116,16.78,86,10.73,2,...,0.061033,-0.147174,0.561314,-0.053201,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
2,57,No,No,0,14.60,127,16.48,93,10.42,2,...,0.053118,-0.147174,0.178109,-0.053201,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
3,58,No,Yes,20,33.07,89,18.16,92,10.65,2,...,0.053118,1.219037,0.561314,0.031576,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956
4,149,No,No,0,31.89,81,17.12,79,11.89,1,...,0.061033,-0.147174,0.561314,0.031576,0.065696,-0.089148,-0.05179,-0.29035,0.007263,0.295956


In [5]:
X = df[[v for v in df.columns if v[:2]=='V_']+[v for v in df.columns if v[:4]=='W_C_']].copy()
y = df['TARGET'].copy()

In [8]:
pipe = make_pipeline(StandardScaler(),PCA(),MinMaxScaler())

In [10]:
pipe.fit(X)
Xs = pd.DataFrame(pipe.transform(X))

In [13]:
Xt,Xv,yt,yv = train_test_split(Xs,y,train_size=0.7)

In [14]:
model = MLPClassifier()

In [15]:
model.fit(Xt,yt)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [16]:
metricas(model,Xt,Xv,yt,yv)

ROC train:0.896 | ROC test:0.911 
ACC train:0.909 | ACC test:0.906 


In [19]:
param_grid = dict(activation = ['identity', 'logistic', 'tanh', 'relu'],
alpha = np.arange(0.0001,0.0005,0.0001),
learning_rate = ['constant', 'invscaling', 'adaptive'],
max_iter = [1000],
hidden_layer_sizes = [(a,b,c,) for a in range(20,35,5) for b in range(20,35,5) for c in range(20,35,5)]
                 )

In [20]:
model = MLPClassifier()

In [28]:
grid = RandomizedSearchCV(cv=3,
                          verbose=True,
                          scoring='roc_auc',
                          estimator=model,
                          n_iter=100,
                          n_jobs=-1,
                          param_distributions=param_grid)

In [29]:
%%time
grid.fit(Xs,y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   56.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.5min finished


CPU times: user 3.15 s, sys: 131 ms, total: 3.28 s
Wall time: 1min 30s


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
          fit_params={}, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'alpha': array([0.0001, 0.0002, 0.0003, 0.0004]), 'learning_rate': ['constant', 'invscaling', 'adaptive'], 'max_iter': [1000], 'activation': ['identity', 'logistic', 'tanh', 'relu'], 'hidden_layer_sizes': [(20, 20, 20), (20, 20, 25), (20, 20, 30), (20, 25, 20), (20, 25, 25), (20..., (30, 20, 30), (30, 25, 20), (30, 25, 25), (30, 25, 30), (30, 30, 20), (30, 30, 25), (30, 30, 30)]},
          pre_dispatch='2*

In [30]:
grid.best_estimator_

MLPClassifier(activation='relu', alpha=0.0004, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(20, 25, 20), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [31]:
model = grid.best_estimator_
model.fit(Xt,yt)

MLPClassifier(activation='relu', alpha=0.0004, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(20, 25, 20), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [32]:
metricas(model,Xt,Xv,yt,yv)

ROC train:0.905 | ROC test:0.910 
ACC train:0.926 | ACC test:0.927 
